Let's start by setting up some simulation parameters and functions. We'll do calculations in WEI to keep close to the original code. We will convert to ETH as necessary. As a preliminary step, let's define some market constants

In [1]:
unitMap = {
    'wei':          1,
    'mwei':         1e6,
    'gwei':         1e9,
    'szabo':        1e12,
    'finney':       1e15,
    'ETH':          1e18,
    'CMT':          1e18
}

def toWei(x, units):
    return x * unitMap[units]

def fromWei(x, units):
    return x / unitMap[units]

Here are the market parameters. The rest of the notebook is designed so that if you change these parameters, the rest of the notebook should execute correctly

In [2]:
PRICE_FLOOR_HUMAN =        0.01 # ETH per CMT
SPREAD =                   110 # 10 %
LIST_REWARD =              toWei(1, 'CMT')
STAKE =                    toWei(10, 'CMT')
# VOTE_BY =                24*3600 # 24 hours.
BACKEND_PAYMENT =          5 # percent
MAKER_PAYMENT =            25 # percent
RESERVE_PAYMENT =          100 - BACKEND_PAYMENT - MAKER_PAYMENT # 70%
COST_PER_BYTE =            toWei(1e-8, 'ETH') # 1 ETH for 100MB

CREATOR_BLOCK_SIZE =       toWei(1e4, 'CMT')
CREATOR_INITIAL_SUPPORT =  toWei(100, 'ETH')

Define market global variables and functions

In [3]:
# Convert human readable floor to units used by market codebase
PRICE_FLOOR = int(PRICE_FLOOR_HUMAN * 1e9) # wei per billion CMT-wei

RESERVE = 0 # Initial reserve is empty
MARKET_TOTAL = 0 # No Market tokens at start

# Note units is ETH-wei per billion CMT-wei
def get_support_price():
    return PRICE_FLOOR + ((SPREAD * RESERVE * 1e9) // (100 * MARKET_TOTAL))

# Human readable support price in ETH per CMT
def get_human_support_price():
    return get_support_price() / 1e9
    
def support(offer):
    support_price = get_support_price()
    MINTED = (offer // support_price) * 1e9 # note GWEI multiplier
    MARKET_TOTAL += MINTED
    RESERVE += offer
    
# differs from contracts in allowing a specified withdraw amount in
# CMT wei units. This is to simplify simulation so that we don't
# have to keep track of a coin table.
def withdraw(amount):
    withdraw_proceeds = (amount * RESERVE) // MARKET_TOTAL
    MARKET_TOTAL = MARKET_TOTAL - amount
    RESERVE = RESERVE - withdraw_proceeds
    
def log():
    print(f"{fromWei(MARKET_TOTAL, 'CMT')} CMT, {fromWei(RESERVE, 'ETH')} ETH")
    print(f"support at {get_human_support_price()} ETH per CMT, withdraw at {RESERVE / MARKET_TOTAL} ETH per CMT")

Ok, now let's define the parameters that govern the current simulation.

In [4]:
CREATOR_SUPPORT = toWei(100, 'ETH')
N_PATRONS = 10
# Support each patron provides
SUPPORT = toWei(1000, 'ETH')
N_MAKERS = 1000
# This is number of listings *per* maker
N_LISTINGS_PER_MAKER = 100
# Size of an individual data purchase in ETH
PURCHASE_SIZE = toWei(1000, 'ETH')
# Number of purchases
N_PURCHASES = 90

Let's have the market creator create the initial data market.

In [5]:
# Creator does their thing
MARKET_TOTAL += CREATOR_BLOCK_SIZE
# Creator deposits funds in reserve
RESERVE += CREATOR_INITIAL_SUPPORT      

In [7]:
log()

10000.0 CMT, 100.0 ETH
support at 0.021 ETH per CMT, withdraw at 0.01 ETH per CMT


In [8]:
get_human_support_price()

0.021

1e+22

In [25]:
x = ((SPREAD * RESERVE * 1e9) // (100 * MARKET_TOTAL))
x = x * 1e-9
x

0.011000000000000001

Let's introduce a data structure to keep track of the coin table.

In [17]:
coin_table = {}
coin_table["CREATOR"] = CREATOR_BLOCK_SIZE